In [1]:
import torch
import matplotlib.pyplot as plt
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.nn import functional as F
from torchvision import transforms

In [2]:
transform = transforms.Compose([transforms.Resize((512,512)),transforms.ToTensor()])

In [3]:
training_data = datasets.Flowers102(root = 'data', split = 'train', download = True, transform = transform)
testing_data = datasets.Flowers102(root = 'data', split = 'test', download = True, transform = transform)

In [4]:
trainingLoader = DataLoader(training_data, batch_size=8, shuffle = True)
testingLoader = DataLoader(testing_data, batch_size=8, shuffle = True)

In [35]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,24,5),
            nn.BatchNorm2d(24),
            nn.ReLU(),
            nn.MaxPool2d(3,3)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(24, 48,5),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.MaxPool2d(3,3)
        )
        self.layer3  = nn.Sequential(
            nn.Conv2d(48,96,5),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(3,3)
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(96,192,5),
            nn.BatchNorm2d(192),
            nn.ReLU(),
            nn.MaxPool2d(3,3)
        )
        self.fc1 = nn.Linear(3072,1000)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(1000,500)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(500,102)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = x.reshape(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

In [36]:
network = NeuralNetwork()

In [37]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr = 0.005)

In [38]:
epoch = 50

for i in range(epoch):
    running_loss = 0
    for j, data in enumerate(trainingLoader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        
        outputs = network(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if j % 100 == 99:
            print(f'[{i + 1}, {j + 1:5d}] loss: {running_loss/100:.5f}')
            running_loss = 0

print('Finished Training')

[1,   100] loss: 4.60167
[2,   100] loss: 4.43136
[3,   100] loss: 4.19943
[4,   100] loss: 3.91881
[5,   100] loss: 3.66767
[6,   100] loss: 3.42319
[7,   100] loss: 3.20766
[8,   100] loss: 3.00126
[9,   100] loss: 2.79311
[10,   100] loss: 2.52762
[11,   100] loss: 2.32606
[12,   100] loss: 2.09825
[13,   100] loss: 1.83190
[14,   100] loss: 1.60117
[15,   100] loss: 1.40724
[16,   100] loss: 1.16692
[17,   100] loss: 0.98527
[18,   100] loss: 0.84248
[19,   100] loss: 0.60192
[20,   100] loss: 0.48661
[21,   100] loss: 0.34849
[22,   100] loss: 0.26566
[23,   100] loss: 0.21357
[24,   100] loss: 0.15085
[25,   100] loss: 0.13461
[26,   100] loss: 0.09969
[27,   100] loss: 0.08398
[28,   100] loss: 0.06474
[29,   100] loss: 0.06080
[30,   100] loss: 0.04898
[31,   100] loss: 0.04716
[32,   100] loss: 0.03982
[33,   100] loss: 0.03303
[34,   100] loss: 0.03634
[35,   100] loss: 0.02959
[36,   100] loss: 0.02646
[37,   100] loss: 0.02448
[38,   100] loss: 0.02540
[39,   100] loss: 0.0

In [39]:
correct = 0
total = 0

for data in testingLoader:
    images, labels = data
    output = network(images)
    _, predicted = torch.max(output.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    
print(f"Network accuracy = {100 * (correct / total)}%")

Network accuracy = 38.28264758497317%
